# Shortest path

In [35]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import json


## Récuperer toutes les gares de France

In [36]:
dfStop = pd.read_csv('../data_sncf/stops.txt', sep=",")
dfStop.shape

(9176, 9)

# New way of calc

## Import file

In [37]:
dfTimeTables = pd.read_csv('../data_sncf/timetables.csv', sep="\t")
dfTimeTables.dropna().head()


,trip_id,trajet,duree
0,OCESN003100F140147152,Gare de Le Havre - Gare de Paris-St-Lazare,138
1,OCESN003190F040047309,Gare de Dieppe - Gare de Paris-St-Lazare,145
2,OCESN003198F030037315,Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...,97
3,OCESN003300F030037323,Gare de Cherbourg - Gare de Paris-St-Lazare,194
4,OCESN003313F380387526,Gare de Caen - Gare de Paris-St-Lazare,149


## Split str to get departure and arrival

## Vérification de l'intégrité des données

In [38]:
# Vérifier si la colonne 'trajet' contient des valeurs nulles
if dfTimeTables['trajet'].isnull().any():
    print("La colonne 'trajet' contient des valeurs nulles.")
else:
    print("La colonne 'trajet' ne contient pas de valeurs nulles.")

# Vérifier si tous les éléments de la colonne 'trajet' ont exactement une occurrence de ' - '
if (dfTimeTables['trajet'].str.count(' - ') == 1).all():
    print("Tous les éléments de la colonne 'trajet' ont exactement une occurrence de ' - '.")
else:
    print("Certains éléments de la colonne 'trajet' n'ont pas exactement une occurrence de ' - '.")

dfTimeTables[dfTimeTables['trajet'].str.count(' - ') != 1]

La colonne 'trajet' ne contient pas de valeurs nulles.
Tous les éléments de la colonne 'trajet' ont exactement une occurrence de ' - '.


,trip_id,trajet,duree


## Split de tous les trajets pour avoir le départ et l'arrivée

In [39]:
dfTimeTables[['depart', 'arrivee']] = dfTimeTables['trajet'].str.split(' - ', expand=True)

dfTimeTables.head()

,trip_id,trajet,duree,depart,arrivee
0,OCESN003100F140147152,Gare de Le Havre - Gare de Paris-St-Lazare,138,Gare de Le Havre,Gare de Paris-St-Lazare
1,OCESN003190F040047309,Gare de Dieppe - Gare de Paris-St-Lazare,145,Gare de Dieppe,Gare de Paris-St-Lazare
2,OCESN003198F030037315,Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...,97,Gare de Paris-St-Lazare,Gare de Rouen-Rive-Droite
3,OCESN003300F030037323,Gare de Cherbourg - Gare de Paris-St-Lazare,194,Gare de Cherbourg,Gare de Paris-St-Lazare
4,OCESN003313F380387526,Gare de Caen - Gare de Paris-St-Lazare,149,Gare de Caen,Gare de Paris-St-Lazare


## Création du graph avec tous les voisins

In [40]:
def get_neighborv1():
    graph = {}
    # graph[dfTimeTables["depart"].values[i]] = {}

    for i in range(len(dfTimeTables["depart"])):
        try:
            depart = dfTimeTables["depart"].values[i]
            arrivee = dfTimeTables["arrivee"].values[i]
            temps = dfTimeTables["duree"].values[i]
            if depart not in graph:
                graph[depart] = {}
            graph[depart] = {**graph[depart], arrivee: int(temps)}

        except Exception as e:
            print("Une erreur s'est produite: {}".format(e))

        # Récupération de tous les noms de gares
        # On part du principe qu'un aller à forcément un retour, on peut donc ne
        # rechercher que dans les départs

    # # create json object from dictionary
    json_values = json.dumps(graph)
    print(json_values)
    #
    now = datetime.datetime.now()

    # Create a file name with the current date and time
    date_string = now.strftime("%Y-%m-%d_%H-%M-%S")
    file_name = "data_" + date_string + ".json"

    # # open file for writing, "w"
    f = open(file_name, "w")
    #
    # # write json object to file
    f.write(json_values)
    #
    # # close file
    f.close()
    print(graph)

# get_neighborv1()

In [41]:
dfStop = dfStop[dfStop['stop_id'].str.contains('StopPoint:OCETrain')]
dfStop.shape

(2447, 9)

In [42]:
allGare = dfStop['stop_name'].unique()
print(allGare)


['Gare de Paris-St-Lazare' 'Gare de Rouen-Rive-Droite' 'Gare de Montville'
 ... 'Gare de Novillars' 'Gare de Deluz' 'Gare de Laissey']


## Extractons nos voisins

In [43]:
dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")

/var/folders/0l/4rhc85qd4kl41w3sd26dml3h0000gn/T/ipykernel_47126/3309677073.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")


In [44]:
def str_to_time(timedate: str):
    time_split = ""
    try:
        time_split = str(timedate).replace('1900-01-01 ','')
    except Exception:
        print(Exception)
        pass
    time_split = time_split.split(':')
    if time_split[0] > '23':
            time_split[0] = str(0)
    time = f'{time_split[0]}:{time_split[1]}:{time_split[2]}'
    # time_object = datetime.strptime(time, '%H:%M:%S')

    return time



In [45]:
# Pre-format date of the dataframe
dfStopTimes['departure_time'] = dfStopTimes['departure_time'].apply(str_to_time)
dfStopTimes['arrival_time'] = dfStopTimes['arrival_time'].apply(str_to_time)
dfStopTimes.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN037071R0100119847,23:05:00,23:05:00,StopPoint:OCECar TER-87381509,0,NaN,0,0,NaN
1,OCESN037071R0100119847,23:35:00,23:35:00,StopPoint:OCECar TER-87415604,1,NaN,0,0,NaN
2,OCESN037071R0100119847,23:55:00,23:55:00,StopPoint:OCECar TER-87415620,2,NaN,0,0,NaN
3,OCESN037071R0100119847,0:25:00,0:25:00,StopPoint:OCECar TER-87415877,3,NaN,0,0,NaN
4,OCESN037071R0100119847,0:45:00,0:45:00,StopPoint:OCECar TER-87411207,4,NaN,0,0,NaN


In [46]:
def getVoisins(gare):
    # on recupere le stopID de notre gare
    stopID = dfStop[(dfStop.stop_name == gare)].stop_id.values[0]
    # on recupere les trajets qui passent par cette gare
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    for i in range(len(trajets)):
        # on recupere la position de la gare dans le trajet
        stopSequence = trajets['stop_sequence'].values[i]
        # on recupere le tripID du trajet
        tripId = trajets['trip_id'].values[i]
        # on recupere les times de notre gare
        departureTimeGare = str_to_time(trajets['departure_time'].values[i])
        arivalTimeGare = str_to_time(trajets['arrival_time'].values[i])

        try:
            # on recupere la gare précédente
            voisin1 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                dfStopTimes['stop_sequence'] == stopSequence - 1)]
            # on recupere le departure time de la gare précédente
            departureTimeVoisin1 = str_to_time(voisin1.departure_time.values[0])
            # on recupere le nom de la gare
            voisin1 = dfStop[dfStop['stop_id'] ==
                             voisin1.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin1] = tempsTrajetvoisin1.seconds
        except:
            continue

        try:
            # on recupere la gare suivante
            voisin2 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                dfStopTimes['stop_sequence'] == stopSequence + 1)]
            # on recupere le departure time de la gare suivante
            arrivalTimeVoisin2 = str_to_time(voisin2.departure_time.values[0])
            # on recupere le nom de la gare
            voisin2 = dfStop[dfStop['stop_id'] ==
                             voisin2.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin2] = tempsTrajetvoisin2.seconds
        except:
            continue

    # on retourne notre liste dans un set pour supprimer les doublons
    return listeVoisinsGare


In [47]:
def getVoisins3(gare):
    stopID = dfStop[(dfStop.stop_name == gare)].stop_id.values[0]
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    for stopSequence, tripId in zip(trajets['stop_sequence'].values, trajets['trip_id'].values):
        try:
            departureTimeGare = str_to_time(trajets['departure_time'].values[i])
            arivalTimeGare = str_to_time(trajets['arrival_time'].values[i])
            mask1 = (dfStopTimes['trip_id'] == tripId) & (dfStopTimes['stop_sequence'] == stopSequence - 1)
            mask2 = (dfStopTimes['trip_id'] == tripId) & (dfStopTimes['stop_sequence'] == stopSequence + 1)

            voisin1 = dfStopTimes[mask1]
            voisin2 = dfStopTimes[mask2]

            if voisin1.size > 0:
                departureTimeVoisin1 = str_to_time(voisin1.departure_time.values[0])
                voisin1Name = dfStop[dfStop['stop_id'] == voisin1.stop_id.values[0]].stop_name.values[0]
                tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
                listeVoisinsGare[voisin1Name] = tempsTrajetvoisin1.seconds

            if voisin2.size > 0:
                arrivalTimeVoisin2 = str_to_time(voisin2.departure_time.values[0])
                voisin2Name = dfStop[dfStop['stop_id'] == voisin2.stop_id.values[0]].stop_name.values[0]
                tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
                listeVoisinsGare[voisin2Name] = tempsTrajetvoisin2.seconds
        except:
            continue
    return listeVoisinsGare

In [48]:
def getVoisins4(gare):
    stopID = dfStop[(dfStop.stop_name == gare)].stop_id.values[0]
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    stopSequences, tripIds = trajets['stop_sequence'].values, trajets['trip_id'].values
    departureTimesGare, arrivalTimesGare = trajets[['departure_time', 'arrival_time']]

    mask1 = (dfStopTimes['trip_id'].values[:, None] == tripIds) & (
                dfStopTimes['stop_sequence'].values[:, None] == stopSequences[None, :] - 1)
    print(mask1)
    mask2 = (dfStopTimes['trip_id'].values[:, None] == tripIds) & (
                dfStopTimes['stop_sequence'].values[:, None] == stopSequences[None, :] + 1)
    print(mask2)

    voisin1 = dfStopTimes[mask1]
    voisin2 = dfStopTimes[mask2]

    voisin1Name = dfStop.loc[voisin1['stop_id'], 'stop_name'].values
    voisin2Name = dfStop.loc[voisin2['stop_id'], 'stop_name'].values
    departureTimesVoisin1, arrivalTimesVoisin2 = voisin1[['departure_time', 'arrival_time']]

    tempsTrajetvoisin1 = arrivalTimesGare[:, None] - departureTimesVoisin1[None, :]
    tempsTrajetvoisin2 = arrivalTimesVoisin2[:, None] - departureTimesGare[None, :]

    listeVoisinsGare.update(dict(zip(voisin1Name, tempsTrajetvoisin1.flatten())))
    listeVoisinsGare.update(dict(zip(voisin2Name, tempsTrajetvoisin2.flatten())))

    return listeVoisinsGare

In [49]:
# def get_neighborv2():
#     graph = {}
#
#     # for i in tqdm(range(len(allGare))):
#     for i in range(10):
#         depart = dfStop["stop_name"].values[i]
#         # On récupère l'id/stop id de notre gare
#         stopID = dfStop[(dfStop.stop_name == allGare[i])].stop_id.values[0]
#
#         # On récupère les trajets qui passent par cette gare
#         trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
#
#         # On parcours ces trajets
#         for j in range(len(trajets)):
#             # on recupere la position de la gare dans le trajet
#             stopSequence = trajets['stop_sequence'].values[j]
#             print(stopSequence)
#             # on recupere le tripID du trajet
#             tripId = trajets['trip_id'].values[j]
#             # graph[depart] = {**graph[depart], voisin: 1}
#
#             # on recupere les times de notre gare
#             # departureTimeGare = str_to_time(trajets['departure_time'].values[j])
#             # arivalTimeGare = str_to_time(trajets['arrival_time'].values[j])
#
#         # on recupere les trajets qui passent par cette gare
#         trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
#         try:
#             print("le")
#             # depart = dfStop['stop_name'].values[i]
#             # if depart not in graph:
#             #     graph[depart] = {}
#             # print(trajets)
#             # for j in range(len(trajets)):
#             #
#             #     graph[depart] = {**graph[depart], trajets[j]: 1}
#         except Exception as e:
#             print("Une erreur s'est produite: {}".format(e))
#
#     print(graph)
#
# #
# # get_neighborv2()

In [50]:
def getVoisins5(gare):
    # on recupere le stopID de notre gare
    stopID = dfStop.loc[dfStop.stop_name == gare, 'stop_id'].iloc[0]
    # on recupere les trajets qui passent par cette gare
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    for j, row in trajets.iterrows():
        # on recupere la position de la gare dans le trajet
        stopSequence = row['stop_sequence']
        # on recupere le tripID du trajet
        tripId = row['trip_id']
        # on recupere les times de notre gare
        departureTimeGare = str_to_time(row['departure_time'])
        arivalTimeGare = str_to_time(row['arrival_time'])

        # on recupere la gare précédente
        voisin1 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                dfStopTimes['stop_sequence'] == stopSequence - 1)]
        try:
            # on recupere le departure time de la gare précédente
            departureTimeVoisin1 = str_to_time(voisin1.departure_time.iloc[0])
            # on recupere le nom de la gare
            voisin1 = dfStop.loc[dfStop['stop_id'] == voisin1.stop_id.iloc[0], 'stop_name'].iloc[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin1] = tempsTrajetvoisin1.seconds
        except:
            continue
        # on recupere la gare suivante
        voisin2 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                dfStopTimes['stop_sequence'] == stopSequence + 1)]
        try:

            # on recupere le arrival time de la gare suivante
            arrivalTimeVoisin2 = str_to_time(voisin2.departure_time.iloc[0])
            # on recupere le nom de la gare
            voisin2 = dfStop.loc[dfStop['stop_id'] == voisin2.stop_id.iloc[0], 'stop_name'].iloc[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin2] = tempsTrajetvoisin2.seconds
        except:
            continue
    # on retourne notre liste dans un set pour supprimer les doublons
    return listeVoisinsGare

In [51]:
def getVoisins6(gare):
    # on recupere le stopID de notre gare
    stopID = dfStop.loc[dfStop.stop_name == gare, 'stop_id'].iloc[0]
    # on recupere les trajets qui passent par cette gare
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    def calc_voisins(row):
        # on recupere la position de la gare dans le trajet
        stopSequence = row['stop_sequence']
        # on recupere le tripID du trajet
        tripId = row['trip_id']
        # on recupere les times de notre gare
        departureTimeGare = str_to_time(row['departure_time'])
        arivalTimeGare = str_to_time(row['arrival_time'])

        # on regroupe les lignes du dataframe par tripId et on décale les valeurs de stop_sequence
        df_voisins = dfStopTimes.groupby('trip_id')
        # print(dfStopTimes.head())
        # on recupere la gare précédente
        voisin1 = df_voisins[(df_voisins['trip_id'] == tripId) & (df_voisins['stop_sequence'] == stopSequence - 1)]
        try:
            # on recupere le departure time de la gare précédente
            departureTimeVoisin1 = str_to_time(voisin1.departure_time.iloc[0])
            # on recupere le nom de la gare
            voisin1 = dfStop.loc[dfStop['stop_id'] == voisin1.stop_id.iloc[0], 'stop_name'].iloc[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin1] = tempsTrajetvoisin1.seconds
        except:
            pass

        # on recupere la gare suivante
        voisin2 = df_voisins[(df_voisins['trip_id'] == tripId) & (df_voisins['stop_sequence'] == stopSequence + 1)]
        try:
            # on recupere le arrival time de la gare suivante
            arrivalTimeVoisin2 = str_to_time(voisin2.departure_time.iloc[0])
            # on recupere le nom de la gare
            voisin2 = dfStop.loc[dfStop['stop_id'] == voisin2.stop_id.iloc[0], 'stop_name'].iloc[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin2] = tempsTrajetvoisin2.seconds
        except:
            pass

    # on applique la fonction aux lignes du dataframe 'trajets'
    trajets.apply(calc_voisins, axis=1)
    # on retourne notre liste dans un set pour supprimer les doublons
    return listeVoisinsGare






In [52]:
def getVoisins7(gare):
    # on recupere le stopID de notre gare
    stopID = dfStop[(dfStop.stop_name == gare)].stop_id.values[0]
    # on recupere les trajets qui passent par cette gare
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    for i in range(len(trajets)):

        # on recupere la position de la gare dans le trajet
        stopSequence = trajets['stop_sequence'].values[i]
        # on recupere le tripID du trajet
        tripId = trajets['trip_id'].values[i]
        # on recupere les times de notre gare
        departureTimeGare = trajets['departure_time'].values[i]
        arivalTimeGare = trajets['arrival_time'].values[i]

        try:
            # on recupere la gare précédente
            voisin1 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                    dfStopTimes['stop_sequence'] == stopSequence - 1)]
            # on recupere le departure time de la gare précédente
            departureTimeVoisin1 = voisin1.departure_time.values[0]
            # on recupere le nom de la gare
            voisin1 = dfStop[dfStop['stop_id'] ==
                             voisin1.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin1] = tempsTrajetvoisin1.seconds
        except:
            continue

        try:
            # on recupere la gare suivante
            voisin2 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                    dfStopTimes['stop_sequence'] == stopSequence + 1)]
            # on recupere le departure time de la gare suivante
            arrivalTimeVoisin2 = voisin2.departure_time.values[0]
            # on recupere le nom de la gare
            voisin2 = dfStop[dfStop['stop_id'] ==
                             voisin2.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin2] = tempsTrajetvoisin2.seconds
        except:
            continue

    # on retourne notre liste dans un set pour supprimer les doublons
    return listeVoisinsGare


In [53]:
graph = {}
# allGare = allGare[['departure_time', 'arrival_time']].apply(str_to_time)
# for i in tqdm(range(len(allGare))):
for i in tqdm(range(10)):
    # for i in tqdm(range(10)):
    graph[allGare[i]] = getVoisins(allGare[i])
print(graph)
# print(getVoisins('Gare de Novillars'))

100%|██████████| 10/10 [00:16<00:00,  1.60s/it]

{'Gare de Paris-St-Lazare': {}, 'Gare de Rouen-Rive-Droite': {}, 'Gare de Montville': {}, 'Gare de Clères': {}, 'Gare de Auffay': {}, 'Gare de Dieppe': {}, 'Gare de Vernon-Giverny': {}, 'Gare de Gaillon-Aubevoye': {}, 'Gare de Val-de-Reuil': {}, 'Gare de Oissel': {}}


In [54]:
print(graph)
# load json module
import json
#
# # create json object from dictionary
json_data = json.dumps(graph)
#

import datetime as date_time_lib
# # Create a file name with the current date and time
# date_string = now.strftime("%Y-%m-%d_%H-%M-%S")
# file_name = "data_" + date_string + ".json"
#
# # # open file for writing, "w"
# f = open(file_name, "w")
# #
# # # write json object to file
# f.write(json_values)
# #
# # # close file
# f.close()
# # open file for writing, "w"
now = date_time_lib.datetime.now()

# Create a file name with the current date and time
date_string = now.strftime("%Y-%m-%d_%H-%M-%S")
file_name = "data_" + date_string + ".json"

# # open file for writing, "w"
f = open(file_name, "w")
#
# # write json object to file
f.write(json_data)
#
# # close file
f.close()

{'Gare de Paris-St-Lazare': {}, 'Gare de Rouen-Rive-Droite': {}, 'Gare de Montville': {}, 'Gare de Clères': {}, 'Gare de Auffay': {}, 'Gare de Dieppe': {}, 'Gare de Vernon-Giverny': {}, 'Gare de Gaillon-Aubevoye': {}, 'Gare de Val-de-Reuil': {}, 'Gare de Oissel': {}}
